### If working with data is part of your daily job, you will likely run into situations where you realize you have to loop through a Pandas Dataframe and process each row.


 ### List of candidates one could potentially try out.
    
    for loop with .iloc
    iterrows
    itertuple
    apply
    python zip
    pandas vectorization
    numpy vectorization

In [58]:
import timeit
import pandas as pd
import numpy as np


In [59]:
# https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.randint.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
    
df = pd.DataFrame(np.random.randint(0, 10, size=(200000, 4)), columns=list('ABCD'))   

In [60]:
df.head(5)

,A,B,C,D
0,3,2,1,6
1,9,4,9,1
2,2,1,2,1
3,7,4,8,6
4,8,5,4,3


In [61]:
df.dtypes

A    int32
B    int32
C    int32
D    int32
dtype: object

### Standard python for loop with iloc

In [62]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html
def loop_with_for(df):
    temp = 0
    for index in range(len(df)):
        temp += df['A'].iloc[index] + df['B'].iloc[index]
    return temp

In [63]:
loop_with_for(df)

1797343

In [64]:
#checking the performance using timeit

%timeit loop_with_for(df)

9.23 s ± 680 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%prun -l 4 loop_with_for(df)

'''Seems like with the for loop + iloc approach, most of the time is spent on accessing values of each cell of the
DataFrame, and checking data type with python’s isinstance function.'''

'Seems like with the for loop + iloc approach, most of the time is spent on accessing values of each cell of the\nDataFrame, and checking data type with python’s isinstance function.'

### Using pandas iterrows function

In [66]:
def loop_with_iterrows(df):
    temp = 0
    for _, row in df.iterrows():
         temp += row.A + row.B
    return temp

loop_with_iterrows(df)

1797343

In [67]:

%timeit loop_with_iterrows(df)

KeyboardInterrupt: 

In [ ]:
%prun -l 4 loop_with_iterrows(df)

'''Surprisingly, the iterrows approach is almost 5 times slow than using standard for loop! The reason, suggested by the log,
is that iterrows spends a lot of time creating pandas Series object, which is known to incur a fair amount of overhead'''

In [ ]:
# Using pandas itertuples function
def loop_with_itertuples(df):
    temp = 0
    for row_tuple in df.itertuples():
        temp += row_tuple.A + row_tuple.B
    return temp

loop_with_itertuples(df)

In [ ]:
%timeit loop_with_itertuples(df)

In [ ]:
%prun -l 4 loop_with_itertuples(df)

'''It is exciting that we are finally getting into miliseconds per loop now! itertuples saves the overhead of creating
Series each row by creating namedtuple instead. This is efficient, yet we are still paying for overhead for creating 
namedtuple.'''

### Using python zip

In [ ]:

#The way it works is it takes a number of iterables, and makes an iterator that aggragates elements from
#each of the iterables. Since a column of a Pandas DataFrame is an iterable, we can utilize zip to produce a tuple
#for each row just like itertuples, without all the pandas overhead!

def loop_with_zip(df):
    temp = 0
    for a, b in zip(df['A'], df['B']):
        temp += a + b
    return temp

loop_with_zip(df)

In [ ]:
%timeit loop_with_zip(df)

In [ ]:
%prun -l 4 loop_with_zip(df)

'''It is about 4 times faster by using zip. The efficiency depends on the fact that we are not creating namedtuple for
every row. zip simply returns an iterator of tuples.'''

### Using pandas apply function


In [ ]:
def using_apply(df):
    return df.apply(lambda x: x['A'] + x['B'], axis=1).sum()

using_apply(df)

In [ ]:
%timeit using_apply(df)

In [ ]:
%prun -l 4 using_apply(df)

'''We are seeing about the same performance as using standard loops. Apply is still doing row operations and creating 
Series, which explains why the function calls are mostly getting values from Series.'''

### Using pandas builtin add function

In [ ]:
def using_pandas_builtin(df):
    return (df['A'] + df['B']).sum()

using_pandas_builtin(df)

In [ ]:
%timeit using_pandas_builtin(df)

In [ ]:
%prun -l 4 using_pandas_builtin(df)

'''pandas built-in add and sum function show how efficient these Pandas built-in functions are. However,many computations
we do does not have a simple built-in operation in Pandas.If the complex operation can be broken down to a series of 
pandas builtin functions, then it might make more sense to go that route than trying to jam all the operations in a 
lambda function then use apply.'''

### Using numpy builtin function

In [ ]:
def using_numpy_builtin(df):
    return (df['A'].values + df['B'].values).sum()

using_numpy_builtin(df)

In [ ]:
%timeit using_numpy_builtin(df)

In [ ]:
%prun -l 4 using_numpy_builtin(df)

'''For efficiency and speed, always go to numpy. Here we convert each column into a numpy array, and does all the heavy 
lifting utilizing numpy’s builtin functionalities. It’s known to be more efficient than Pandas Dataframe operations'''

### What about DataFrame with different size?

#### Now we have a good understanding of the efficiency of each approach, a natural question to ask is: how large the DataFrame needs to be for us to start considering trying a more efficient approach? 

#### To answer that question,an experiment on DataFrames with different sizes, from 1000 rows to 40000 rows was performed. 

Iterrows is the least efficient and computation time grows the fastest. Then on second tier we have the apply function and standard for loop, which have almost same performance. Then the **most efficient** approaches are                      **pandas and numpy built-in functions**, the performance of which are very consistent despite increasing number of rows. Close seconds are zip and itertuples, with zip approach slightly faster than itertuples. 

#### References: https://medium.com/swlh/how-to-efficiently-loop-through-pandas-dataframe-660e4660125d

